# 修饰器

## 无参情况
### 无修饰器函数传参例子：

In [34]:
def hi():
    """hi func，假装是很复杂的函数"""
    return 'hi'

def aop(func):
    """aop func"""
    print('before func')
    print(func())
    print('after func')
    
if __name__ == '__main__':
    aop(hi)

before func
hi
after func


### 修饰器装饰

In [35]:
def aop(func):
    """aop func"""
    def wrapper():
        """wrapper func"""
        print('before func')
        func()
        print('after func')
    return wrapper

@aop
def hi():
    """hi func"""
    print('hi')

if __name__ == '__main__':
    hi()

before func
hi
after func


简单来说，换言之：`@aop 等效于hi = aop(hi)`。对于@，可以理解是，它通过闭包的方式把新函数的引用赋值给了原来函数的引用

## 含参情况

In [36]:
from functools import wraps

def aop(func):
    """aop func"""
    @wraps(func)
    def wrap(*args, **kwargs):
        print('before')
        func(*args, **kwargs)
        print('after')

    return wrap

@aop
def hi(a, b, c):
    """hi func"""
    print('test hi: %s, %s, %s' % (a, b, c))

@aop
def hello(a, b):
    """hello func"""
    print('test hello: %s, %s' % (a, b))

if __name__ == '__main__':
    hi(1, 2, 3)
    hello('a', 'b')

before
test hi: 1, 2, 3
after
before
test hello: a, b
after


### 带参修饰器

In [37]:
from functools import wraps

def aop_with_param(aop_test_str):
    def aop(func):
        """aop func"""
        @wraps(func)
        def wrap(*args, **kwargs):
            print('before ' + str(aop_test_str))
            func(*args, **kwargs)
            print('after ' + str(aop_test_str))
        return wrap
    return aop

@aop_with_param('abc')
def hi(a, b, c):
    """hi func"""
    print('test hi: %s, %s, %s' % (a, b, c))

@aop_with_param('pppppp')
def hi2(a, b, c):
    """hi func"""
    print('test hi: %s, %s, %s' % (a, b, c))

if __name__ == '__main__':
    hi(1, 2, 3)
    print('')
    hi2(2, 3, 4)

before abc
test hi: 1, 2, 3
after abc

before pppppp
test hi: 2, 3, 4
after pppppp


## 修饰器类

In [38]:
from functools import wraps

class aop(object):
    def __init__(self, aop_test_str):
        self.aop_test_str = aop_test_str

    def __call__(self, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print('before ' + self.aop_test_str)
            func()
            print('after ' + self.aop_test_str)

        return wrapper
        
@aop('pppppp')
def hi():
    print('hi')

if __name__=='__main__':
    hi()

before pppppp
hi
after pppppp


### 继承修饰器

In [39]:
class sub_aop(aop): # 继承aop父类
    def __init__(self, sub_aop_str, *args, **kwargs):
        self.sub_aop_str = sub_aop_str
        super(sub_aop, self).__init__(*args, **kwargs)

# 所有的函数都是可调用对象。
#一个类实例也可以变成一个可调用对象，只需要实现一个特殊方法__call__()。
# 意思是可以直接调用类，而不需要另外写一个方法来调用类了。
# 所以当sub_aop继承aop类时，aop中的__call__方法让aop类被调用，输出aop中__call__的内容
    def __call__(self, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print('before ' + self.sub_aop_str)
            super(sub_aop, self).__call__(func)()
            print('after ' + self.sub_aop_str)
        return wrapper
        
@sub_aop('ssssss', 'pppppp')
def hello():
    print('hello')
    
if __name__ == '__main__':
    hello()

before ssssss
before pppppp
hello
after pppppp
after ssssss


#### 对上述代码的深入理解：

In [40]:
class sub_aop(aop): # 继承aop父类
    def __init__(self, sub_aop_str, *args, **kwargs):
        self.sub_aop_str = sub_aop_str
        super(sub_aop, self).__init__(*args, **kwargs)

    # def __call__(self, func):
    #     @wraps(func)
    #     def wrapper(*args, **kwargs):
    #         print('before ' + self.sub_aop_str)
    #         super(sub_aop, self).__call__(func)()
    #         print('after ' + self.sub_aop_str)
    #     return wrapper
        
@sub_aop('ssssss', 'pppppp')
def hello():
    print('hello')
    
if __name__ == '__main__':
    hello()

'''
@sub_aop('ssssss', 'pppppp')的入参，'ssssss'赋予sub_aop()中的sub_aop_str, 'pppppp'赋予*args并在使用super()时调用父类aop()的__init__方法时
赋予aop()的aop_test_str, aop()自动调用__call__()方法输入如下

'''

before pppppp
hello
after pppppp


"\n@sub_aop('ssssss', 'pppppp')的入参，'ssssss'赋予sub_aop()中的sub_aop_str, 'pppppp'赋予*args并在使用super()时调用父类aop()的__init__方法时\n赋予aop()的aop_test_str, aop()自动调用__call__()方法输入如下\n\n"

In [41]:
class sub_aop(aop): # 继承aop父类
    def __init__(self, sub_aop_str, *args, **kwargs):
        self.sub_aop_str = sub_aop_str
        super(sub_aop, self).__init__(*args, **kwargs)

    def __call__(self, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print('before ' + self.sub_aop_str)
            # super(sub_aop, self).__call__(func)()
            print('after ' + self.sub_aop_str)
        return wrapper
        
@sub_aop('ssssss', 'pppppp')
def hello():
    print('hello')
    
if __name__ == '__main__':
    hello()

'''
@因sub_aop()重写了aop(), 所以不调用sub_aop的__call__方法, 输出如下

'''

before ssssss
after ssssss


'\n@因sub_aop()重写了aop(), 所以不调用sub_aop的__call__方法, 输出如下\n\n'

In [43]:
class sub_aop(aop): # 继承aop父类
    def __init__(self, sub_aop_str, *args, **kwargs):
        self.sub_aop_str = sub_aop_str
        super(sub_aop, self).__init__(*args, **kwargs)

    def __call__(self, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print('before ' + self.sub_aop_str)
            super(sub_aop, self).__call__(func)()
            print('after ' + self.sub_aop_str)
        return wrapper
        
@sub_aop('ssssss', 'pppppp')
def hello():
    print('hello')
    
if __name__ == '__main__':
    hello()

'''
@因sub_aop()调用父类的__call__方法, 传入func参数, 输出如下

'''

before ssssss
before pppppp
hello
after pppppp
after ssssss


'\n@因sub_aop()调用父类的__call__方法, 传入func参数, 输出如下\n\n'

# 关键字用法

*args 表示任何多个无名参数， 他本质上是一个 tuple

** kwargs 表示关键字参数， 它本质上是一个 dict

参考：https://blog.csdn.net/GODSuner/article/details/117961990

> 所以上述代码去掉`**kwargs`并无影响。

# super()的用法

In [19]:
class Father(): 
    def __init__(self, house=10, *args, **kwargs): #老爷子花了10w买的房
        self.house = house 
        print('father have house')                                                                                                                                               

class Son(Father): 
    def __init__(self, car=5, *args, **kwargs): #儿子花了5w买的车
        super(Son, self).__init__() #调用父类Father的函数__init__，
        self.car = car 
        print('son have car')                                                                                                                                              

if __name__=='__main__':
    son = Son(5)
    print((son.house, son.car))

father have house
son have car
(10, 5)


类的继承中，如定义了与继承父类的同名方法`__init__()`；该方法希望能继承实现父类的功能，同时能进行扩展，那此时需要调用父类的方法，如何调用？super进行实现（继承调用父类方法的功能）。

参考：https://zhuanlan.zhihu.com/p/641440838

# 单双下划线

- 单下划线：
  - 在Python中，单下划线作为命名约定，用于表示某个变量、方法或属性是内部使用的或具有特定含义，但不希望在外部直接访问。
- 双下划线：
  - 名称修饰。双下划线用于名称修饰，即将属性或方法前面的双下划线作为前缀，使其成为私有成员。私有成员只能在类内部访问，无法在类外部直接访问。

参考：https://zhuanlan.zhihu.com/p/640538185#:~:text=%E5%8D%95%E4%B8%8B%E5%88%92%E7%BA%BF%E4%BD%9C%E4%B8%BA%E5%91%BD%E5%90%8D%E7%BA%A6%E5%AE%9A%E3%80%81%E9%81%BF%E5%85%8D%E5%91%BD%E5%90%8D%E5%86%B2%E7%AA%81%E3%80%81%E4%B8%B4%E6%97%B6%E5%8F%98%E9%87%8F%E5%92%8C%E5%BC%95%E5%85%A5%E6%9C%AA%E4%BD%BF%E7%94%A8%E7%9A%84%E5%8F%98%E9%87%8F%E7%AD%89%E6%96%B9%E9%9D%A2%E5%8F%91%E6%8C%A5%E4%BD%9C%E7%94%A8%E3%80%82,%E5%8F%8C%E4%B8%8B%E5%88%92%E7%BA%BF%E7%94%A8%E4%BA%8E%E5%90%8D%E7%A7%B0%E4%BF%AE%E9%A5%B0%E3%80%81%E5%90%8D%E7%A7%B0%E6%94%B9%E5%86%99%E5%92%8C%E5%9C%A8%E9%AD%94%E6%9C%AF%E6%96%B9%E6%B3%95%E4%B8%AD%E6%89%AE%E6%BC%94%E9%87%8D%E8%A6%81%E8%A7%92%E8%89%B2%E3%80%82%20%E6%8E%8C%E6%8F%A1%E5%8D%95%E4%B8%8B%E5%88%92%E7%BA%BF%E5%92%8C%E5%8F%8C%E4%B8%8B%E5%88%92%E7%BA%BF%E7%9A%84%E4%BD%BF%E7%94%A8%EF%BC%8C%E5%B0%86%E5%B8%AE%E5%8A%A9%E4%BD%A0%E6%9B%B4%E5%A5%BD%E5%9C%B0%E7%90%86%E8%A7%A3%E5%92%8C%E5%BA%94%E7%94%A8Python%E7%BC%96%E7%A8%8B%E4%B8%AD%E7%9A%84%E5%91%BD%E5%90%8D%E7%BA%A6%E5%AE%9A%E3%80%81%E4%BB%A3%E7%A0%81%E7%BB%84%E7%BB%87%E5%92%8C%E5%AF%B9%E8%B1%A1%E8%A1%8C%E4%B8%BA%E5%AE%9A%E5%88%B6%E3%80%82